In [65]:
#1
from typing import List, Dict


def multiset_intersection(a: List[int], b: List[int]) -> List[int]:
    
    if len(a) > len(b):
        a, b = b, a

    counts: Dict[int, int] = {}
    for x in a:
        counts[x] = counts.get(x, 0) + 1

    result: List[int] = []
    for x in b:
        
        if counts.get(x, 0) > 0:
            result.append(x)
            counts[x] -= 1

    return result


if __name__ == "__main__":
    A = [1, 2, 2, 3, 5]
    B = [2, 2, 2, 3, 4]
    print(multiset_intersection(A, B))  

[2, 2, 3]


In [ ]:
#2
import random
from typing import List, Dict, Optional


def text_to_words(text: str) -> List[str]:
   
    words: List[str] = []
    current: List[str] = []

    for ch in text:
        if ch.isalpha():
            current.append(ch.lower())
        else:
            if current:
                words.append("".join(current))
                current.clear()

    if current:
        words.append("".join(current))

    return words


def build_bigrams(words: List[str]) -> Dict[str, List[str]]:
    
    bigrams: Dict[str, List[str]] = {}

    for i in range(len(words) - 1):
        w1 = words[i]
        w2 = words[i + 1]
        if w1 not in bigrams:
            bigrams[w1] = []
        bigrams[w1].append(w2)

    return bigrams


def autocomplete(bigrams: Dict[str, List[str]], start_word: str, add_count: int = 4) -> Optional[str]:

    start_word = start_word.lower()

    if start_word not in bigrams:
        return None

    result_words = [start_word]
    current = start_word

    for _ in range(add_count):
        next_options = bigrams.get(current)
        if not next_options:
            break
        nxt = random.choice(next_options)
        result_words.append(nxt)
        current = nxt

    return " ".join(result_words)


def task2_main(path_to_text_file: str) -> None:
    # 1 читаем текст
    try:
        with open(path_to_text_file, "r", encoding="utf-8") as f:
            text = f.read()
    except FileNotFoundError:
        print("Ошибка: файл не найден.")
        return

    # 2 текст -> слова
    words = text_to_words(text)
    print(f"Слов в тексте: {len(words)}")
    print(f"Пример первых 10 слов: {words[:10]}")

    # 3 слова -> биграммы
    bigrams = build_bigrams(words)
    print(f"Уникальных ключей (первых слов биграмм): {len(bigrams)}")
    print(f"Пример первых 3 биграмм: {list(bigrams.items())[:3]}")

   
    while True:
        w = input("Введите слово (или пустую строку для выхода): ").strip()
        if not w:
            break

        phrase = autocomplete(bigrams, w, add_count=4)  
        if phrase is None:
            print("Слово не найдено")
        else:
            print(f"Автодополненная фраза: {phrase}")


if __name__ == "__main__":
    task2_main("text.txt")  

Слов в тексте: 43
Пример первых 10 слов: ['python', 'is', 'a', 'high', 'level', 'general', 'purpose', 'programming', 'language', 'its']
Уникальных ключей (первых слов биграмм): 38
Пример первых 3 биграмм: [('python', ['is', 'is']), ('is', ['a', 'dynamically']), ('a', ['high'])]
Автодополненная фраза: is dynamically type checked and
Автодополненная фраза: python is dynamically type checked
Автодополненная фраза: high level general purpose programming
Автодополненная фраза: language its design philosophy emphasizes
Слово не найдено
Автодополненная фраза: including structured particularly procedural object
Автодополненная фраза: including structured particularly procedural object


In [ ]:
# 3.1. Хеш-функция
def word_hash(s: str, m: int) -> int:
    p = 1_000_000_007  #простое число для модульного деления
    base = 911382323  #основание для хеширования
    h = 0
    for ch in s:
        h = (h * base + (ord(ch) + 1)) % p  # (ord(ch) + 1) для избегания нулевого значения
    return h % m

#хеш-таблица с цепочками
from typing import List

class HashSetWords:
    def __init__(self, m: int = 10007):
        self.m = m  #размер хеш-таблицы
        self.buckets: List[List[str]] = [[] for _ in range(m)]  #массив корзин
        self.collisions = 0  #счетчик коллизий
        self.size = 0  #счетчик уникальных элементов

    def _idx(self, word: str) -> int:
        return word_hash(word, self.m)

    def add(self, word: str) -> None:
        word = word.lower()
        idx = self._idx(word)
        bucket = self.buckets[idx]

        if word not in bucket:
            if len(bucket) > 0:  #если корзина не пуста, то была коллизия
                self.collisions += 1
            bucket.append(word)
            self.size += 1

    def contains(self, word: str) -> bool:
        word = word.lower()
        idx = self._idx(word)
        return word in self.buckets[idx]

#интерактивный режим
def task3_interactive() -> None:
    hs = HashSetWords(m=10007)  #создаем экземпляр хеш-таблицы

    while True:
        line = input("Введите команду (add/check) или пустую строку для выхода: ").strip()
        if not line:
            break  #выход из программы

        parts = line.split(maxsplit=1)
        if len(parts) != 2:
            print("Неверная команда")
            continue

        cmd, word = parts[0], parts[1]

        if cmd == "add":
            hs.add(word)
            print(f"Слово '{word}' добавлено.")
        elif cmd == "check":
            if hs.contains(word):
                print("yes")
            else:
                print("no")
        else:
            print("Неверная команда")


task3_interactive()  

Слово 'python' добавлено.
yes
no


In [64]:
# 3.2 
def word_hash(s: str, m: int) -> int:
    p = 1_000_000_007  #простое число для модульного деления
    base = 911382323  #основание для хеширования
    h = 0
    for ch in s:
        h = (h * base + (ord(ch) + 1)) % p  # (ord(ch) + 1) для избегания нулевого значения
    return h % m

#хеш-таблица с цепочками
from typing import List

class HashSetWords:
    def __init__(self, m: int = 10007):
        self.m = m  #размер хеш-таблицы
        self.buckets: List[List[str]] = [[] for _ in range(m)]  #массив корзин
        self.collisions = 0  #счетчик коллизий
        self.size = 0  #счетчик уникальных элементов

    def _idx(self, word: str) -> int:
        return word_hash(word, self.m)

    def add(self, word: str) -> None:
        word = word.lower()
        idx = self._idx(word)
        bucket = self.buckets[idx]

        if word not in bucket:
            if len(bucket) > 0:  #если корзина не пуста, то была коллизия
                self.collisions += 1
            bucket.append(word)
            self.size += 1

    def contains(self, word: str) -> bool:
        word = word.lower()
        idx = self._idx(word)
        return word in self.buckets[idx]

#  оценка идеальных коллизий
def estimate_ideal_collisions(n: int, m: int) -> float:
    """
    Идеальная оценка коллизий при равномерном распределении:
    ideal_collisions ≈ n - E[occupied]
    """
    occupied = m * (1 - (1 - 1 / m) ** n)
    return n - occupied

#основная функция для обработки текста и вычисления коллизий
def task3_collisions_on_text(text: str, m: int = 10007) -> None:
    words = text.lower().split()  #разбиваем текст на слова

    hs = HashSetWords(m=m)  #создаем хеш-таблицу
    for word in words:
        hs.add(word)

    real_collisions = hs.collisions  #реальные коллизии

    ideal_collisions = estimate_ideal_collisions(hs.size, m)  #идеальные коллизии

    print(f"Размер хеш-таблицы m = {m}")
    print(f"Количество уникальных слов: {hs.size}")
    print(f"Реальные коллизии: {real_collisions}")
    print(f"Идеальные коллизии (оценка): {ideal_collisions:.2f}")
    print(f"Средняя длина корзины: {hs.size / m:.4f}")


text = """
Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant indentation. Python is dynamically type-checked and garbage-collected. It supports multiple programming paradigms, including structured (particularly procedural), object-oriented and functional programming.
"""


task3_collisions_on_text(text)

Размер хеш-таблицы m = 10007
Количество уникальных слов: 34
Реальные коллизии: 0
Идеальные коллизии (оценка): 0.06
Средняя длина корзины: 0.0034
